# Import all libraries possibly needed

In [ ]:
import pandas as pd
import numpy as np
# Import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'../')
%load_ext autoreload
%autoreload 2

import datetime 
from datetime import datetime as dt
from geopy.distance import vincenty # requires separate install - pip install geopy
import warnings
warnings.filterwarnings('ignore')

#Imports for Classification
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB #Naive Bayes Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#Split Training and Testing Set.
from sklearn.model_selection import train_test_split

#Generate Classification Performance Results
from sklearn.metrics import classification_report

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#Import imputer
from sklearn.preprocessing import Imputer

# statistic analysis
from sklearn.feature_selection import f_classif
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

# Load in data

In [2]:
# Load in the data with `read_csv()`
Charlotte_11_15_data = pd.read_csv("Datasets/Charlotte-Mecklenburg_2011-2015.csv", header = 0)
Charlotte_16_data = pd.read_csv("Datasets/Charlotte-Mecklenburg_2016.csv", header = 0)

# Define Classes

In [3]:
# define number of classes
NUM_OF_CLASS = 5

# interval values to retrieve crime rate from crime rate classes
interval_vcr = max(Charlotte_11_15_data['Violent_Crime_Rate_Log2']) / NUM_OF_CLASS + 0.000001
interval_pcr = max(Charlotte_11_15_data['Property_Crime_Rate_Log2']) / NUM_OF_CLASS + 0.000001

# add columns for classes
Charlotte_11_15_data['Violent_Crime_Rate_Class'] = 0
Charlotte_11_15_data['Property_Crime_Rate_Class'] = 0


for index, row in Charlotte_11_15_data.iterrows():
    if(row['Violent_Crime_Rate'] != -1):
        Charlotte_11_15_data.at[index, 'Violent_Crime_Rate_Class'] = int(Charlotte_11_15_data.at[index, 'Violent_Crime_Rate_Log2'] / interval_vcr)
    else:
        Charlotte_11_15_data.at[index, 'Violent_Crime_Rate_Class'] = -1
      
    if(row['Property_Crime_Rate'] != -1):
        Charlotte_11_15_data.at[index, 'Property_Crime_Rate_Class'] = int(Charlotte_11_15_data.at[index, 'Property_Crime_Rate_Log2'] / interval_pcr)
    else:
        Charlotte_11_15_data.at[index, 'Property_Crime_Rate_Class'] = -1

# See what the data is like

In [4]:
# Get the keys of the `digits` data
print(Charlotte_11_15_data.keys())

# 1386 rows, 37 cols
print(Charlotte_11_15_data.shape)

# violent crime rate class
Charlotte_11_15_data_Violent_Crime_Rate_Class = Charlotte_11_15_data.Violent_Crime_Rate_Class
print((np.unique(Charlotte_11_15_data_Violent_Crime_Rate_Class)))

# property crime rate class
Charlotte_11_15_data_data_Property_Crime_Rate_Class = Charlotte_11_15_data.Property_Crime_Rate_Class
print((np.unique(Charlotte_11_15_data_data_Property_Crime_Rate_Class)))

Index([u'NPA', u'Vacant_Land', u'Vacant_Land_Area', u'Commercial_Construction',
       u'Commercial_Construction_Permitted_Units', u'Commercial_Size',
       u'Commercial_Size_Total', u'Commercial_Building_Age',
       u'Impervious_Surface', u'Impervious_Surface_Area', u'Adopt_a_Stream',
       u'Adopt_a_Stream_Length', u'Pharmacy_Proximity',
       u'Pharmacy_Proximate_Units', u'Housing_Density', u'Housing_Units',
       u'Single_Family_Housing', u'Single_Family_Units', u'Housing_Size',
       u'Housing_Age', u'New_Residential', u'New_Residential_Permit_Units',
       u'Residential_Renovation', u'Residential_Renovation_Permit_Units',
       u'Foreclosures', u'Foreclosed_Units', u'Housing_Violations',
       u'Housing_Violations_Total', u'Street_Connectivity',
       u'Transit_Proximity', u'Transit_Proximate_Units', u'Violent_Crime_Rate',
       u'Property_Crime_Rate', u'Combined_Crime_Rate',
       u'Violent_Crime_Rate_Log2', u'Property_Crime_Rate_Log2',
       u'Combined_Crime_Rate_L

In [5]:
# Get the keys of the `digits` data
print(Charlotte_16_data.keys())

# 462 rows, 31 cols
print(Charlotte_16_data.shape)


Index([u'NPA', u'Vacant_Land', u'Vacant_Land_Area', u'Commercial_Construction',
       u'Commercial_Construction_Permitted_Units', u'Commercial_Size',
       u'Commercial_Size_Total', u'Commercial_Building_Age',
       u'Impervious_Surface', u'Impervious_Surface_Area', u'Adopt_a_Stream',
       u'Adopt_a_Stream_Length', u'Pharmacy_Proximity',
       u'Pharmacy_Proximate_Units', u'Housing_Density', u'Housing_Units',
       u'Single_Family_Housing', u'Single_Family_Units', u'Housing_Size',
       u'Housing_Age', u'New_Residential', u'New_Residential_Permit_Units',
       u'Residential_Renovation', u'Residential_Renovation_Permit_Units',
       u'Foreclosures', u'Foreclosed_Units', u'Housing_Violations',
       u'Housing_Violations_Total', u'Street_Connectivity',
       u'Transit_Proximity', u'Transit_Proximate_Units'],
      dtype='object')
(462, 31)


# Change -1 to NaN in the dataframe

In [6]:
# https://machinelearningmastery.com/handle-missing-data-python/

# replace missing values with NaN
Charlotte_11_15_data[:] = Charlotte_11_15_data[:].replace(-1, np.NaN)

Charlotte_16_data[:] = Charlotte_16_data[:].replace(-1, np.NaN)

# If we simply ignore missing values

In [7]:
# If we simply drop missing values
Charlotte_11_15_data_trimmed = Charlotte_11_15_data.copy(deep=True)
Charlotte_16_data_trimmed = Charlotte_16_data.copy(deep=True)

Charlotte_11_15_data_trimmed.dropna(inplace=True, how='any')
Charlotte_16_data_trimmed.dropna(inplace=True, how='any')

In [8]:
# only Violent Crime Rate
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_trimmed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_trimmed.loc[:,['Violent_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_vcr = MLPClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Naive Bayes Classifier')
%time c_predict_vcr = GaussianNB().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Decision Tree Classifier')
%time c_predict_vcr = DecisionTreeClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('K Nearest Neighbors')
%time c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_vcr, c_train_vcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 14 ms
             precision    recall  f1-score   support

        0.0       0.23      0.39      0.29        80
        1.0       0.00      0.00      0.00         0
        2.0       0.58      0.20      0.30       141
        3.0       0.00      0.00      0.00        29
        4.0       0.00      0.00      0.00         7

avg / total       0.39      0.23      0.25       257

Naive Bayes Classifier
Wall time: 0 ns
             precision    recall  f1-score   support

        0.0       0.76      0.79      0.77       127
        1.0       0.15      0.36      0.22        22
        2.0       0.54      0.37      0.44        70
        3.0       0.50      0.33      0.40        36
        4.0       0.00      0.00      0.00         2

avg / total       0.61      0.57      0.58       257

Decision Tree Classifier
Wall time: 15 ms
             precision    recall  f1-score   support

        0.0       0.77      0.81      0.79       125
        1.0  

In [9]:
# only Property Crime Rate
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_trimmed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_trimmed.loc[:,['Property_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_pcr = MLPClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Naive Bayes Classifier')
%time c_predict_pcr = GaussianNB().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Decision Tree Classifier')
%time c_predict_pcr = DecisionTreeClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('K Nearest Neighbors')
%time c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_pcr, c_train_pcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 22 ms
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         0
        1.0       0.79      0.53      0.64       133
        2.0       0.61      0.71      0.66       124
        3.0       0.00      0.00      0.00         0
        4.0       0.00      0.00      0.00         0

avg / total       0.70      0.62      0.65       257

Naive Bayes Classifier
Wall time: 2 ms
             precision    recall  f1-score   support

        0.0       0.20      0.08      0.11        13
        1.0       0.77      0.43      0.55       159
        2.0       0.27      0.78      0.40        50
        3.0       0.59      0.29      0.38        35
        4.0       0.00      0.00      0.00         0

avg / total       0.62      0.46      0.48       257

Decision Tree Classifier
Wall time: 14 ms
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         4
        1.0  

In [10]:
# Change the size of training dataset to build a better model
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_trimmed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_trimmed.loc[:,['Violent_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_trimmed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_trimmed.loc[:,['Property_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)

# predict crime rate besed on best classifier from above
c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(Charlotte_16_data_trimmed)
c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(Charlotte_16_data_trimmed)

Charlotte_16_data_write2csv = Charlotte_16_data_trimmed.copy(deep=True)

In [11]:
Charlotte_16_data_write2csv['Violent_Crime_Rate_Class'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Class'] = c_predict_pcr

Charlotte_16_data_write2csv['Combined_Crime_Rate'] = 0.0

Charlotte_16_data_write2csv['Violent_Crime_Rate_Log2'] = Charlotte_16_data_write2csv['Violent_Crime_Rate_Class'] * interval_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Log2'] = Charlotte_16_data_write2csv['Property_Crime_Rate_Class'] * interval_pcr

Charlotte_16_data_write2csv['Violent_Crime_Rate'] = pow(Charlotte_16_data_write2csv['Violent_Crime_Rate_Log2'], 2.0)
Charlotte_16_data_write2csv['Property_Crime_Rate'] = pow(Charlotte_16_data_write2csv['Property_Crime_Rate_Log2'], 2.0)

In [12]:
# reorder columns to keep consistent with 11-15 csv file
Charlotte_16_data_write2csv['vcr_temp'] = Charlotte_16_data_write2csv['Violent_Crime_Rate']
Charlotte_16_data_write2csv['pcr_temp'] = Charlotte_16_data_write2csv['Property_Crime_Rate']

Charlotte_16_data_write2csv['Violent_Crime_Rate'] = Charlotte_16_data_write2csv['Violent_Crime_Rate_Class']
Charlotte_16_data_write2csv['Property_Crime_Rate'] = Charlotte_16_data_write2csv['Property_Crime_Rate_Class']

Charlotte_16_data_write2csv['Violent_Crime_Rate_Class'] = Charlotte_16_data_write2csv['vcr_temp']
Charlotte_16_data_write2csv['Property_Crime_Rate_Class'] = Charlotte_16_data_write2csv['pcr_temp']

Charlotte_16_data_write2csv.drop('vcr_temp', axis=1, inplace=True)
Charlotte_16_data_write2csv.drop('pcr_temp', axis=1, inplace=True)

# Don't forget to rename columns!
Charlotte_16_data_write2csv.rename(columns={'Violent_Crime_Rate_Class': 'newName1', 'Property_Crime_Rate_Class': 'newName2'}, inplace=True)
Charlotte_16_data_write2csv.rename(columns={'Violent_Crime_Rate': 'Violent_Crime_Rate_Class', 'Property_Crime_Rate': 'Property_Crime_Rate_Class'}, inplace=True)
Charlotte_16_data_write2csv.rename(columns={'newName1':'Violent_Crime_Rate', 'newName2':'Property_Crime_Rate'}, inplace=True)

for index, row in Charlotte_16_data_write2csv.iterrows():
    Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = (Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] + Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate']) / 2.0

In [13]:
# write to csv file
Charlotte_16_data_write2csv.to_csv("Datasets/Charlotte-Mecklenburg_2016_predict_IGNORE.csv", sep=',', encoding='utf-8')

In [ ]:
# Plot bar charts based on predicted data for 2016
c_predict_2016_vcr = c_predict_vcr.astype(int)
c_predict_2016_pcr = c_predict_pcr.astype(int)

count_vcr_2016 = np.bincount(c_predict_2016_vcr)
count_pcr_2016 = np.bincount(c_predict_2016_pcr)

# print c_predict_2016_vcr
# print c_predict_2016_pcr

# Count # of class predictions 
count_class_0_vcr_2016 = count_vcr_2016[0]
count_class_1_vcr_2016 = count_vcr_2016[1]
count_class_2_vcr_2016 = count_vcr_2016[2]
count_class_3_vcr_2016 = count_vcr_2016[3]

count_class_0_pcr_2016 = count_pcr_2016[0]
count_class_1_pcr_2016 = count_pcr_2016[1]
count_class_2_pcr_2016 = count_pcr_2016[2]
count_class_3_pcr_2016 = count_pcr_2016[3]

# Define Classes Names
#objects = ("Class 0: " + str(count_class_0_vcr_2016), 'Class 1', 'Class 2', 'Class 3')
objects = ('Lower Crime', '', '', 'Higher Crime')
y_pos = np.arange(len(objects))

# Plotting Violent Crime Data 2016
performance = [count_class_0_vcr_2016, count_class_1_vcr_2016, count_class_2_vcr_2016, count_class_3_vcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Violent Crime')
plt.title('Violent Crime Data 2016 - Ignore')
plt.savefig('Barcharts/Violent_Crime_Data_2016_IGNORE.png')
plt.show()

# Plotting Property Crime Data 2016
performance = [count_class_0_pcr_2016, count_class_1_pcr_2016, count_class_2_pcr_2016, count_class_3_pcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Property Crime')
plt.title('Property Crime Data 2016 - Ignore')
plt.savefig('Barcharts/Property_Crime_Data_2016_IGNORE.png')
plt.show()


# If we impute missing values

# I. Assign fixed value to missing one

In [14]:
# Replacing missing values with -1
Charlotte_11_15_data_fixed = Charlotte_11_15_data.copy(deep=True)
Charlotte_16_data_fixed = Charlotte_16_data.copy(deep=True)

Charlotte_11_15_data_fixed.fillna(10, inplace = True)
Charlotte_16_data_fixed.fillna(10, inplace = True)

In [15]:
# only Violent Crime Rate
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_fixed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_fixed.loc[:,['Violent_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_vcr = MLPClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Naive Bayes Classifier')
%time c_predict_vcr = GaussianNB().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Decision Tree Classifier')
%time c_predict_vcr = DecisionTreeClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('K Nearest Neighbors')
%time c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_vcr, c_train_vcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 61 ms
             precision    recall  f1-score   support

       -1.0       0.00      0.00      0.00         7
        0.0       0.48      0.66      0.56       157
        1.0       0.31      0.22      0.26       149
        2.0       0.45      0.27      0.34       145
        3.0       0.00      0.00      0.00         0
        4.0       0.00      0.00      0.00         0

avg / total       0.41      0.38      0.38       458

Naive Bayes Classifier
Wall time: 0 ns
             precision    recall  f1-score   support

       -1.0       0.12      0.08      0.10        25
        0.0       0.74      0.75      0.75       213
        1.0       0.10      0.37      0.16        30
        2.0       0.43      0.42      0.42        88
        3.0       0.77      0.24      0.37        96
        4.0       1.00      0.50      0.67         6

avg / total       0.61      0.52      0.53       458

Decision Tree Classifier
Wall time: 16 ms
             p

In [16]:
# only Property Crime Rate
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_fixed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_fixed.loc[:,['Property_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_pcr = MLPClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Naive Bayes Classifier')
%time c_predict_pcr = GaussianNB().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Decision Tree Classifier')
%time c_predict_pcr = DecisionTreeClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('K Nearest Neighbors')
%time c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_pcr, c_train_pcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 25 ms
             precision    recall  f1-score   support

       -1.0       0.00      0.00      0.00         0
        0.0       0.00      0.00      0.00         0
        1.0       0.30      0.60      0.40        83
        2.0       0.92      0.54      0.68       374
        3.0       0.00      0.00      0.00         1
        4.0       0.00      0.00      0.00         0

avg / total       0.80      0.55      0.63       458

Naive Bayes Classifier
Wall time: 0 ns
             precision    recall  f1-score   support

       -1.0       0.17      0.30      0.21        10
        0.0       0.48      0.08      0.14       123
        1.0       0.54      0.48      0.51       186
        2.0       0.25      0.75      0.38        75
        3.0       0.70      0.33      0.45        63
        4.0       0.00      0.00      0.00         1

avg / total       0.49      0.39      0.37       458

Decision Tree Classifier
Wall time: 16 ms
             p

In [17]:
# Change the size of training dataset to build a better model
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_fixed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_fixed.loc[:,['Violent_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_fixed.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_fixed.loc[:,['Property_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)

# predict crime rate besed on best classifier from above
c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(Charlotte_16_data_fixed)
c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(Charlotte_16_data_fixed)

Charlotte_16_data_write2csv = Charlotte_16_data_fixed.copy(deep=True)

In [18]:
Charlotte_16_data_write2csv['Violent_Crime_Rate'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate'] = c_predict_pcr

Charlotte_16_data_write2csv['Combined_Crime_Rate'] = 0.0

Charlotte_16_data_write2csv['Violent_Crime_Rate_Log2'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Log2'] = c_predict_pcr

Charlotte_16_data_write2csv['Violent_Crime_Rate_Class'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Class'] = c_predict_pcr

for index, row in Charlotte_16_data_write2csv.iterrows():
    if(row['Violent_Crime_Rate_Class'] != 10):
        Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Class'] * interval_vcr
        Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'], 2.0)
        
    if(row['Property_Crime_Rate_Class'] != 10):
        Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Class'] * interval_vcr
        Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'], 2.0)

for index, row in Charlotte_16_data_write2csv.iterrows():
    if(row['Violent_Crime_Rate'] != 10 and row['Property_Crime_Rate'] != 10):
        Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = (Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] + Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate']) / 2.0
    elif (row['Violent_Crime_Rate'] != 10):
        Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate']
    elif (row['Property_Crime_Rate'] != 10):
          Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate']
    else:
      Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = 10
        
Charlotte_16_data_write2csv[:] = Charlotte_16_data_write2csv[:].replace(10, np.NaN)   

In [19]:
# write to csv file
Charlotte_16_data_write2csv.to_csv("Datasets/Charlotte-Mecklenburg_2016_predict_FIXED.csv", sep=',', encoding='utf-8')

In [ ]:
# Plot bar charts based on predicted data for 2016
c_predict_2016_vcr = c_predict_vcr.astype(int)
c_predict_2016_pcr = c_predict_pcr.astype(int)

count_vcr_2016 = np.bincount(c_predict_2016_vcr)
count_pcr_2016 = np.bincount(c_predict_2016_pcr)

# print c_predict_2016_vcr
# print c_predict_2016_pcr

# Count # of class predictions 
count_class_0_vcr_2016 = count_vcr_2016[0]
count_class_1_vcr_2016 = count_vcr_2016[1]
count_class_2_vcr_2016 = count_vcr_2016[2]
count_class_3_vcr_2016 = count_vcr_2016[3]

count_class_0_pcr_2016 = count_pcr_2016[0]
count_class_1_pcr_2016 = count_pcr_2016[1]
count_class_2_pcr_2016 = count_pcr_2016[2]
count_class_3_pcr_2016 = count_pcr_2016[3]

# Define Classes Names
#objects = ("Class 0: " + str(count_class_0_vcr_2016), 'Class 1', 'Class 2', 'Class 3')
objects = ('Lower Crime', '', '', 'Higher Crime')
y_pos = np.arange(len(objects))

# Plotting Violent Crime Data 2016
performance = [count_class_0_vcr_2016, count_class_1_vcr_2016, count_class_2_vcr_2016, count_class_3_vcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Violent Crime')
plt.title('Violent Crime Data 2016 - Fixed')
plt.savefig('Barcharts/Violent_Crime_Data_2016_FIXED.png')
plt.show()

# Plotting Property Crime Data 2016
performance = [count_class_0_pcr_2016, count_class_1_pcr_2016, count_class_2_pcr_2016, count_class_3_pcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Property Crime')
plt.title('Property Crime Data 2016 - Fixed')
plt.savefig('Barcharts/Property_Crime_Data_2016_FIXED.png')
plt.show()


# II. Imputer() Mean

In [20]:
# Impute missing values with means
Charlotte_11_15_data_imputed_mean = Charlotte_11_15_data.copy(deep=True)
Charlotte_16_data_imputed_mean = Charlotte_16_data.copy(deep=True)

cols_11_15 = Charlotte_11_15_data_imputed_mean.columns
cols_16 = Charlotte_16_data_imputed_mean.columns

imputer_11_15 = Imputer(strategy='mean')
Charlotte_11_15_data_imputed_mean = imputer_11_15.fit_transform(Charlotte_11_15_data_imputed_mean)
Charlotte_11_15_data_imputed_mean = pd.DataFrame(Charlotte_11_15_data_imputed_mean, columns = cols_11_15)

imputer_16 = Imputer(strategy='mean')
Charlotte_16_data_imputed_mean = imputer_16.fit_transform(Charlotte_16_data_imputed_mean)
Charlotte_16_data_imputed_mean = pd.DataFrame(Charlotte_16_data_imputed_mean, columns = cols_16)

# make sure class values are integer
for index, row in Charlotte_11_15_data_imputed_mean.iterrows():
    Charlotte_11_15_data_imputed_mean.at[index, 'Violent_Crime_Rate_Class'] = int(Charlotte_11_15_data_imputed_mean.at[index, 'Violent_Crime_Rate_Class'])
    Charlotte_11_15_data_imputed_mean.at[index, 'Property_Crime_Rate_Class'] = int(Charlotte_11_15_data_imputed_mean.at[index, 'Property_Crime_Rate_Class'])

In [21]:
# only Violent Crime Rate
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_imputed_mean.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_mean.loc[:,['Violent_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_vcr = MLPClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Naive Bayes Classifier')
%time c_predict_vcr = GaussianNB().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Decision Tree Classifier')
%time c_predict_vcr = DecisionTreeClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('K Nearest Neighbors')
%time c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_vcr, c_train_vcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 28 ms
             precision    recall  f1-score   support

        0.0       0.81      0.54      0.65       395
        1.0       0.05      0.06      0.05        63
        2.0       0.00      0.00      0.00         0
        3.0       0.00      0.00      0.00         0
        4.0       0.00      0.00      0.00         0

avg / total       0.70      0.47      0.57       458

Naive Bayes Classifier
Wall time: 2 ms
             precision    recall  f1-score   support

        0.0       0.70      0.92      0.79       200
        1.0       0.36      0.32      0.34        98
        2.0       0.41      0.50      0.45        68
        3.0       0.83      0.22      0.35        90
        4.0       0.00      0.00      0.00         2

avg / total       0.61      0.59      0.56       458

Decision Tree Classifier
Wall time: 16 ms
             precision    recall  f1-score   support

        0.0       0.83      0.83      0.83       261
        1.0  

In [22]:
# only Property Crime Rate
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_imputed_mean.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_mean.loc[:,['Property_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_pcr = MLPClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Naive Bayes Classifier')
%time c_predict_pcr = GaussianNB().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Decision Tree Classifier')
%time c_predict_pcr = DecisionTreeClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('K Nearest Neighbors')
%time c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_pcr, c_train_pcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 52 ms
             precision    recall  f1-score   support

        0.0       0.05      0.01      0.02       103
        1.0       0.12      0.67      0.20        33
        2.0       0.67      0.53      0.59       292
        3.0       0.00      0.00      0.00         0
        4.0       0.00      0.00      0.00        30

avg / total       0.45      0.39      0.40       458

Naive Bayes Classifier
Wall time: 1e+03 µs
             precision    recall  f1-score   support

        0.0       0.16      0.07      0.10        43
        1.0       0.65      0.54      0.59       223
        2.0       0.52      0.73      0.61       162
        3.0       0.50      0.42      0.46        26
        4.0       1.00      0.25      0.40         4

avg / total       0.55      0.56      0.54       458

Decision Tree Classifier
Wall time: 22 ms
             precision    recall  f1-score   support

        0.0       0.16      0.16      0.16        19
        1

In [23]:
# Change the size of training dataset to build a better model
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_imputed_mean.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_mean.loc[:,['Violent_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_imputed_mean.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_mean.loc[:,['Property_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)

# predict crime rate besed on best classifier from above
c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(Charlotte_16_data_imputed_mean)
c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(Charlotte_16_data_imputed_mean)

Charlotte_16_data_write2csv = Charlotte_16_data.copy(deep=True)

In [24]:
Charlotte_16_data_write2csv['Violent_Crime_Rate'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate'] = c_predict_pcr

Charlotte_16_data_write2csv['Combined_Crime_Rate'] = 0.0

Charlotte_16_data_write2csv['Violent_Crime_Rate_Log2'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Log2'] = c_predict_pcr

Charlotte_16_data_write2csv['Violent_Crime_Rate_Class'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Class'] = c_predict_pcr

for index, row in Charlotte_16_data_write2csv.iterrows():
    Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Class'] * interval_vcr
    Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'], 2.0)
        
    Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Class'] * interval_vcr
    Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'], 2.0)
    
    Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = (Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] + Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate']) / 2.0

In [25]:
# write to csv file
Charlotte_16_data_write2csv.to_csv("Datasets/Charlotte-Mecklenburg_2016_predict_MEAN.csv", sep=',', encoding='utf-8')

In [ ]:
# Plot bar charts based on predicted data for 2016
c_predict_2016_vcr = c_predict_vcr.astype(int)
c_predict_2016_pcr = c_predict_pcr.astype(int)

count_vcr_2016 = np.bincount(c_predict_2016_vcr)
count_pcr_2016 = np.bincount(c_predict_2016_pcr)

# print c_predict_2016_vcr
# print c_predict_2016_pcr

# Count # of class predictions 
count_class_0_vcr_2016 = count_vcr_2016[0]
count_class_1_vcr_2016 = count_vcr_2016[1]
count_class_2_vcr_2016 = count_vcr_2016[2]
count_class_3_vcr_2016 = count_vcr_2016[3]

count_class_0_pcr_2016 = count_pcr_2016[0]
count_class_1_pcr_2016 = count_pcr_2016[1]
count_class_2_pcr_2016 = count_pcr_2016[2]
count_class_3_pcr_2016 = count_pcr_2016[3]

# Define Classes Names
#objects = ("Class 0: " + str(count_class_0_vcr_2016), 'Class 1', 'Class 2', 'Class 3')
objects = ('Lower Crime', '', '', 'Higher Crime')
y_pos = np.arange(len(objects))

# Plotting Violent Crime Data 2016
performance = [count_class_0_vcr_2016, count_class_1_vcr_2016, count_class_2_vcr_2016, count_class_3_vcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Violent Crime')
plt.title('Violent Crime Data 2016 - Mean')
plt.savefig('Barcharts/Violent_Crime_Data_2016_MEAN.png')
plt.show()

# Plotting Property Crime Data 2016
performance = [count_class_0_pcr_2016, count_class_1_pcr_2016, count_class_2_pcr_2016, count_class_3_pcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Property Crime')
plt.title('Property Crime Data 2016 - Mean')
plt.savefig('Barcharts/Property_Crime_Data_2016_MEAN.png')
plt.show()


# III. Imputer() Median

In [26]:
# Impute missing values with median
Charlotte_11_15_data_imputed_median = Charlotte_11_15_data.copy(deep=True)
Charlotte_16_data_imputed_median = Charlotte_16_data.copy(deep=True)

cols_11_15 = Charlotte_11_15_data_imputed_median.columns
cols_16 = Charlotte_16_data_imputed_median.columns

imputer_11_15 = Imputer(strategy='median')
Charlotte_11_15_data_imputed_median = imputer_11_15.fit_transform(Charlotte_11_15_data_imputed_median)
Charlotte_11_15_data_imputed_median = pd.DataFrame(Charlotte_11_15_data_imputed_median, columns = cols_11_15)

imputer_16 = Imputer(strategy='median')
Charlotte_16_data_imputed_median = imputer_16.fit_transform(Charlotte_16_data_imputed_median)
Charlotte_16_data_imputed_median = pd.DataFrame(Charlotte_16_data_imputed_median, columns = cols_16)

# make sure class values are integer
for index, row in Charlotte_11_15_data_imputed_median.iterrows():
    Charlotte_11_15_data_imputed_median.at[index, 'Violent_Crime_Rate_Class'] = int(Charlotte_11_15_data_imputed_median.at[index, 'Violent_Crime_Rate_Class'])
    Charlotte_11_15_data_imputed_median.at[index, 'Property_Crime_Rate_Class'] = int(Charlotte_11_15_data_imputed_median.at[index, 'Property_Crime_Rate_Class'])

In [27]:
# only Violent Crime Rate
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_imputed_median.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_median.loc[:,['Violent_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_vcr = MLPClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Naive Bayes Classifier')
%time c_predict_vcr = GaussianNB().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Decision Tree Classifier')
%time c_predict_vcr = DecisionTreeClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('K Nearest Neighbors')
%time c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_vcr, c_train_vcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 46 ms
             precision    recall  f1-score   support

        0.0       0.84      0.64      0.73       335
        1.0       0.08      0.25      0.12        28
        2.0       0.00      0.00      0.00         0
        3.0       0.62      0.19      0.29        86
        4.0       0.00      0.00      0.00         9

avg / total       0.73      0.52      0.59       458

Naive Bayes Classifier
Wall time: 2 ms
             precision    recall  f1-score   support

        0.0       0.73      0.89      0.80       213
        1.0       0.23      0.28      0.25        74
        2.0       0.41      0.48      0.44        67
        3.0       0.81      0.21      0.34        98
        4.0       0.00      0.00      0.00         6

avg / total       0.61      0.57      0.55       458

Decision Tree Classifier
Wall time: 17 ms
             precision    recall  f1-score   support

        0.0       0.83      0.83      0.83       258
        1.0  

In [28]:
# only Property Crime Rate
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_imputed_median.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_median.loc[:,['Property_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_pcr = MLPClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Naive Bayes Classifier')
%time c_predict_pcr = GaussianNB().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Decision Tree Classifier')
%time c_predict_pcr = DecisionTreeClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('K Nearest Neighbors')
%time c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_pcr, c_train_pcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 41 ms
             precision    recall  f1-score   support

        0.0       0.24      0.07      0.11        69
        1.0       0.50      0.41      0.45       214
        2.0       0.37      0.49      0.42       175
        3.0       0.00      0.00      0.00         0
        4.0       0.00      0.00      0.00         0

avg / total       0.41      0.39      0.39       458

Naive Bayes Classifier
Wall time: 1e+03 µs
             precision    recall  f1-score   support

        0.0       0.33      0.15      0.20        48
        1.0       0.77      0.55      0.64       244
        2.0       0.25      0.77      0.38        77
        3.0       0.74      0.23      0.35        87
        4.0       0.00      0.00      0.00         2

avg / total       0.63      0.48      0.50       458

Decision Tree Classifier
Wall time: 20 ms
             precision    recall  f1-score   support

        0.0       0.33      0.33      0.33        21
        1

In [29]:
# Change the size of training dataset to build a better model
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_imputed_median.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_median.loc[:,['Violent_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_imputed_median.loc[:, ['NPA', 'Vacant_Land', 
    'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_median.loc[:,['Property_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)

# predict crime rate besed on best classifier from above
c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(Charlotte_16_data_imputed_median)
c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(Charlotte_16_data_imputed_median)

Charlotte_16_data_write2csv = Charlotte_16_data.copy(deep=True)

In [30]:
Charlotte_16_data_write2csv['Violent_Crime_Rate'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate'] = c_predict_pcr

Charlotte_16_data_write2csv['Combined_Crime_Rate'] = 0.0

Charlotte_16_data_write2csv['Violent_Crime_Rate_Log2'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Log2'] = c_predict_pcr

Charlotte_16_data_write2csv['Violent_Crime_Rate_Class'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Class'] = c_predict_pcr

for index, row in Charlotte_16_data_write2csv.iterrows():
    Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Class'] * interval_vcr
    Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'], 2.0)
        
    Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Class'] * interval_vcr
    Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'], 2.0)
    
    Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = (Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] + Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate']) / 2.0

In [31]:
# write to csv file
Charlotte_16_data_write2csv.to_csv("Datasets/Charlotte-Mecklenburg_2016_predict_MEDIAN.csv", sep=',', encoding='utf-8')

In [ ]:
# Plot bar charts based on predicted data for 2016
c_predict_2016_vcr = c_predict_vcr.astype(int)
c_predict_2016_pcr = c_predict_pcr.astype(int)

count_vcr_2016 = np.bincount(c_predict_2016_vcr)
count_pcr_2016 = np.bincount(c_predict_2016_pcr)

# print c_predict_2016_vcr
# print c_predict_2016_pcr

# Count # of class predictions 
count_class_0_vcr_2016 = count_vcr_2016[0]
count_class_1_vcr_2016 = count_vcr_2016[1]
count_class_2_vcr_2016 = count_vcr_2016[2]
count_class_3_vcr_2016 = count_vcr_2016[3]

count_class_0_pcr_2016 = count_pcr_2016[0]
count_class_1_pcr_2016 = count_pcr_2016[1]
count_class_2_pcr_2016 = count_pcr_2016[2]
count_class_3_pcr_2016 = count_pcr_2016[3]

# Define Classes Names
#objects = ("Class 0: " + str(count_class_0_vcr_2016), 'Class 1', 'Class 2', 'Class 3')
objects = ('Lower Crime', '', '', 'Higher Crime')
y_pos = np.arange(len(objects))

# Plotting Violent Crime Data 2016
performance = [count_class_0_vcr_2016, count_class_1_vcr_2016, count_class_2_vcr_2016, count_class_3_vcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Violent Crime')
plt.title('Violent Crime Data 2016 - Median')
plt.savefig('Barcharts/Violent_Crime_Data_2016_MEDIAN.png')
plt.show()

# Plotting Property Crime Data 2016
performance = [count_class_0_pcr_2016, count_class_1_pcr_2016, count_class_2_pcr_2016, count_class_3_pcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Property Crime')
plt.title('Property Crime Data 2016 - Median')
plt.savefig('Barcharts/Property_Crime_Data_2016_MEDIAN.png')
plt.show()


# IV. Imputer() Most Frequently Used

In [32]:
# Impute missing values with most_frequent
Charlotte_11_15_data_imputed_most_frequent = Charlotte_11_15_data.copy(deep=True)
Charlotte_16_data_imputed_most_frequent = Charlotte_16_data.copy(deep=True)

cols_11_15 = Charlotte_11_15_data_imputed_most_frequent.columns
cols_16 = Charlotte_16_data_imputed_most_frequent.columns

imputer_11_15 = Imputer(strategy='most_frequent')
Charlotte_11_15_data_imputed_most_frequent = imputer_11_15.fit_transform(Charlotte_11_15_data_imputed_most_frequent)
Charlotte_11_15_data_imputed_most_frequent = pd.DataFrame(Charlotte_11_15_data_imputed_most_frequent, columns = cols_11_15)

imputer_16 = Imputer(strategy='most_frequent')
Charlotte_16_data_imputed_most_frequent = imputer_16.fit_transform(Charlotte_16_data_imputed_most_frequent)
Charlotte_16_data_imputed_most_frequent = pd.DataFrame(Charlotte_16_data_imputed_most_frequent, columns = cols_16)

# make sure class values are integer
for index, row in Charlotte_11_15_data_imputed_most_frequent.iterrows():
    Charlotte_11_15_data_imputed_most_frequent.at[index, 'Violent_Crime_Rate_Class'] = int(Charlotte_11_15_data_imputed_most_frequent.at[index, 'Violent_Crime_Rate_Class'])
    Charlotte_11_15_data_imputed_most_frequent.at[index, 'Property_Crime_Rate_Class'] = int(Charlotte_11_15_data_imputed_most_frequent.at[index, 'Property_Crime_Rate_Class'])

In [33]:
# only Violent Crime Rate
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_imputed_most_frequent.loc[:, ['NPA', 
    'Vacant_Land', 'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_most_frequent.loc[:,['Violent_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_vcr = MLPClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Naive Bayes Classifier')
%time c_predict_vcr = GaussianNB().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Decision Tree Classifier')
%time c_predict_vcr = DecisionTreeClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('K Nearest Neighbors')
%time c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(f_test_vcr)
print(classification_report(c_predict_vcr, c_test_vcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_vcr, c_train_vcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 53 ms
             precision    recall  f1-score   support

        0.0       1.00      0.53      0.69       458
        1.0       0.00      0.00      0.00         0
        2.0       0.00      0.00      0.00         0
        3.0       0.00      0.00      0.00         0
        4.0       0.00      0.00      0.00         0

avg / total       1.00      0.53      0.69       458

Naive Bayes Classifier
Wall time: 1e+03 µs
             precision    recall  f1-score   support

        0.0       0.69      0.88      0.77       190
        1.0       0.26      0.30      0.28        83
        2.0       0.53      0.42      0.47       112
        3.0       0.62      0.29      0.40        69
        4.0       1.00      0.25      0.40         4

avg / total       0.57      0.57      0.55       458

Decision Tree Classifier
Wall time: 18 ms
             precision    recall  f1-score   support

        0.0       0.83      0.84      0.84       240
        1

In [34]:
# only Property Crime Rate
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_imputed_most_frequent.loc[:, ['NPA', 
    'Vacant_Land', 'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_most_frequent.loc[:,['Property_Crime_Rate_Class']], test_size=0.33, train_size =0.67, shuffle=True)

print('Multi-Layer Perceptron Classifier')
%time c_predict_pcr = MLPClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Naive Bayes Classifier')
%time c_predict_pcr = GaussianNB().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Decision Tree Classifier')
%time c_predict_pcr = DecisionTreeClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('K Nearest Neighbors')
%time c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(f_test_pcr)
print(classification_report(c_predict_pcr, c_test_pcr))

print('Linear Discriminant Analysis (LDA)')
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, f_train_pcr, c_train_pcr, cv=kfold, scoring='accuracy')
print(result.mean())

Multi-Layer Perceptron Classifier
Wall time: 153 ms
             precision    recall  f1-score   support

        0.0       0.26      0.13      0.17        39
        1.0       0.33      0.64      0.44        87
        2.0       0.89      0.65      0.75       332
        3.0       0.00      0.00      0.00         0
        4.0       0.00      0.00      0.00         0

avg / total       0.73      0.60      0.64       458

Naive Bayes Classifier
Wall time: 1e+03 µs
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        20
        1.0       0.84      0.44      0.58       324
        2.0       0.20      0.72      0.32        68
        3.0       0.71      0.42      0.53        40
        4.0       1.00      0.33      0.50         6

avg / total       0.70      0.46      0.51       458

Decision Tree Classifier
Wall time: 17 ms
             precision    recall  f1-score   support

        0.0       0.11      0.22      0.14         9
        

In [35]:
# Change the size of training dataset to build a better model
f_train_vcr, f_test_vcr, c_train_vcr, c_test_vcr = train_test_split(Charlotte_11_15_data_imputed_most_frequent.loc[:, ['NPA', 
    'Vacant_Land', 'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_most_frequent.loc[:,['Violent_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)
f_train_pcr, f_test_pcr, c_train_pcr, c_test_pcr = train_test_split(Charlotte_11_15_data_imputed_most_frequent.loc[:, ['NPA', 
    'Vacant_Land', 'Vacant_Land_Area', 'Commercial_Construction', 'Commercial_Construction_Permitted_Units', 'Commercial_Size', 
    'Commercial_Size_Total', 'Commercial_Building_Age', 'Impervious_Surface', 'Impervious_Surface_Area', 'Adopt_a_Stream', 
    'Adopt_a_Stream_Length', 'Pharmacy_Proximity', 'Pharmacy_Proximate_Units', 'Housing_Density', 'Housing_Units', 
    'Single_Family_Housing', 'Single_Family_Units', 'Housing_Size', 'Housing_Age', 'New_Residential', 'New_Residential_Permit_Units', 
    'Residential_Renovation', 'Residential_Renovation_Permit_Units', 'Foreclosures', 'Foreclosed_Units', 'Housing_Violations', 
    'Housing_Violations_Total', 'Street_Connectivity', 'Transit_Proximity', 'Transit_Proximate_Units']], 
    Charlotte_11_15_data_imputed_most_frequent.loc[:,['Property_Crime_Rate_Class']], test_size=0.01, train_size = 0.99, shuffle=True)

# predict crime rate besed on best classifier from above
c_predict_vcr = KNeighborsClassifier().fit(f_train_vcr, c_train_vcr).predict(Charlotte_16_data_imputed_most_frequent)
c_predict_pcr = KNeighborsClassifier().fit(f_train_pcr, c_train_pcr).predict(Charlotte_16_data_imputed_most_frequent)

Charlotte_16_data_write2csv = Charlotte_16_data.copy(deep=True)

In [36]:
Charlotte_16_data_write2csv['Violent_Crime_Rate'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate'] = c_predict_pcr

Charlotte_16_data_write2csv['Combined_Crime_Rate'] = 0.0

Charlotte_16_data_write2csv['Violent_Crime_Rate_Log2'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Log2'] = c_predict_pcr

Charlotte_16_data_write2csv['Violent_Crime_Rate_Class'] = c_predict_vcr
Charlotte_16_data_write2csv['Property_Crime_Rate_Class'] = c_predict_pcr

for index, row in Charlotte_16_data_write2csv.iterrows():
    Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Class'] * interval_vcr
    Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate_Log2'], 2.0)
        
    Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'] = Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Class'] * interval_vcr
    Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate'] = pow(Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate_Log2'], 2.0)
    
    Charlotte_16_data_write2csv.at[index, 'Combined_Crime_Rate'] = (Charlotte_16_data_write2csv.at[index, 'Violent_Crime_Rate'] + Charlotte_16_data_write2csv.at[index, 'Property_Crime_Rate']) / 2.0

In [37]:
# write to csv file
Charlotte_16_data_write2csv.to_csv("Datasets/Charlotte-Mecklenburg_2016_predict_MOST_FREQUENT.csv", sep=',', encoding='utf-8')

In [ ]:
# Plot bar charts based on predicted data for 2016
c_predict_2016_vcr = c_predict_vcr.astype(int)
c_predict_2016_pcr = c_predict_pcr.astype(int)

count_vcr_2016 = np.bincount(c_predict_2016_vcr)
count_pcr_2016 = np.bincount(c_predict_2016_pcr)

# print c_predict_2016_vcr
# print c_predict_2016_pcr

# Count # of class predictions 
count_class_0_vcr_2016 = count_vcr_2016[0]
count_class_1_vcr_2016 = count_vcr_2016[1]
count_class_2_vcr_2016 = count_vcr_2016[2]
count_class_3_vcr_2016 = count_vcr_2016[3]

count_class_0_pcr_2016 = count_pcr_2016[0]
count_class_1_pcr_2016 = count_pcr_2016[1]
count_class_2_pcr_2016 = count_pcr_2016[2]
count_class_3_pcr_2016 = count_pcr_2016[3]

# Define Classes Names
#objects = ("Class 0: " + str(count_class_0_vcr_2016), 'Class 1', 'Class 2', 'Class 3')
objects = ('Lower Crime', '', '', 'Higher Crime')
y_pos = np.arange(len(objects))

# Plotting Violent Crime Data 2016
performance = [count_class_0_vcr_2016, count_class_1_vcr_2016, count_class_2_vcr_2016, count_class_3_vcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Violent Crime')
plt.title('Violent Crime Data 2016 - Most Frequent')
plt.savefig('Barcharts/Violent_Crime_Data_2016_MOST_FREQUENT.png')
plt.show()

# Plotting Property Crime Data 2016
performance = [count_class_0_pcr_2016, count_class_1_pcr_2016, count_class_2_pcr_2016, count_class_3_pcr_2016]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Property Crime')
plt.title('Property Crime Data 2016 - Most Frequent')
plt.savefig('Barcharts/Property_Crime_Data_2016_MOST_FREQUENT.png')
plt.show()
